# Miniature/spotaneous postsynaptic currents
Recording and analyzing miniature/spontaneous postsynaptic currents (m/sPSCs) is one of the most common experiments in patch clamp electrophysiology. m/sPSCs are ionic currents from AMPA, NMDA, glycine, or GABAA receptors that are evoked due to release of a single or multiple synaptic vesicles. We will cover both mPSCs and sPSCs, and go over how to analyze PSC events. While this chapter focuses on PSCs, most of the theory applies to miniature/spontaneous postsynaptic potentials (PSPs) as well. There are several experiments you can do that utilize PSCs; synapse number, silent synapses, excitatory/inhibitory ratio, synaptic multiplicity, and changes in synaptic release regulated by other non-ionic receptors.

## Miniature postsynaptic currents
Miniature postsynaptic currents (mPSCs) are ionic currents evoked from the release of a single synaptic vesicle {cite}`del_castillo_quantal_1954`. Frequency of mPSCs is used as a proxy for the number of functional synapses (synapses that have presynaptic input) that contain the receptor of interest (but not necessarily the number of synapses). The interpretation of mPSC data depends on what receptor you are recording from. If you are recording mEPSCs from AMPARs then you are likely getting the number of "active" or "non-silent" synapses. If you are recording NMDARs could be looking at the number of silent synapses (only if you compare to the AMPAR mEPSCs). If you are recording mIPSCs then you are getting the number of inhibitory synapses. With mIPSCs you could be getting GABAAR or GlyR. One important caveat of mPSCs is that you are not getting where the presynaptic input is coming from. If you want projection specific synaptic input you need to run a different type of experiment than we will be covering here, but is covered later in the book.

### Internal and external solutions
You will need to block spontaneous activity including tetrodotoxin (TTX) in the bath. Preferably you would also use an internal solution that contains cesium and QX-314. For more information on internals see {doc}`./internal_solutions.ipynb`. Depending on the receptor current(s) you want to record you will need to block certain other receptors by including specific drugs in the external solutions. For more information on externals see {doc}`./external_solutions.ipynb`.

### How should you record mPSCs
mPSCs are currents which means you are recording in voltage-clamp mode. This means that the amplifier will injecct current into the cell to keep it at your choosen holding voltage. Any changes in current means that the cells had a change in voltage that the amplifier is counter acting

There are two primary ways you can record mPSCs. One way is you can record continuously for about 3-5 minutes. Technically speaking this is the easiest method since you have a single recording and pretty much any simple recording software will implement this method. The second way is you can record 20-40 sweeps/acquisitions of 5-15 seconds each. Each of these acquisitions act as a kind of technical replicate. This method allows you to discard bad portions of the recording. Usually when you get proficient at patching you will rarely have bad recordings however sometimes you get 30 seconds where there is an unstable seal, digital cell phone noise, your bath gets too low, you get pump/vacuum noise or other issues. When this happens it is fine to discard the 5 or so acquisitions that are bad. You can create acquisitions from continuous recordings by splitting to get the same benefits of the sweeps/acquisitions method.

Filtering and sample rate are the other important consideration for capturing mPSCs. You generally want the sample rate to be 3-4x greater than the filter cutoff you are using with the filter cutoff determining the high frequency you are interested in. While signal theory says you have to have the sample rate 2x greater than the high frequency you are interested in, generally to capture that high frequency well you need to sample 3-4x times that rate. In general mPSCs are recorded at a 10000 Hz with a 3000 Hz lowpass cutoff. A 10000 Hz sample rate is high enough to capture the rise of a mPSCs which are fairly quick, especially for mEPSCs on parvalbumin interneurons. 10000 Hz is also a good trade off between accuracy of the signal and digital storage space. In modern times you could realistically record at 20000 Hz with no storage space issues. I suggest a mininum sample rate of 10000 Hz.

## Analyzing miniature/spontaneous PSCs.
There are several important features of m/aPSCs that you will want to analyze. The primary feature is frequency (the number of events occuring every second). Frequency is used as a proxy for the number of synapses that contain the receptor whose currents you are recording. The more mPSCs, the more synapses. However, if there are changes in release probability you could also get a change in frequency without a change in synapse number. To determine whether there are changes in release probability you will need to run some pair-pulse experiments which are described in a later chapter. Another feature is mPSC amplitude. A larger amplitude could mean two things. Larger mPSCs could mean there are more receptors at the postsynaptic element. Alternatively, larger mPSCs could mean that you get less distal synapses due to decreased dendritic length or more synapses close to the cell body with no change in dendritic length (probably less likely to occur). Amplitude of mPSCs decreases the further from the cell body that the event occurs due to dendritic filtering. To rule out one of the interpretations you can use the mEPSC rise rate (amplitude/peak_time-baseline_start) to determine if the rise rate is changed. More distal mPSCs should have a slower rise rate due to dendritic lowpass filtering. Lastly, you can look at the tau or decay rate of the mPSC. Changes in tau are usually due to changes in receptor subunit composition. Tau is especially useful when you need to identify specific cells types. Interneurons, like parvalbumin interneurons, have a very short mPESC tau compared to pyramidal neurons. Tau can also be affected by dendritic filtering.

First we are going to import some python packages.

In [ ]:
from pathlib import Path
import json

from ipywidgets import widgets, HBox, VBox, Label, FloatSlider
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats, signal, optimize

Next we are going to load the data. All the data is stored on json files. While this file type is not the most practical for storing electrophysiological data, it is the very convenient since it does not require any third party python packages.

In [ ]:
path = Path().cwd() 
path = path.parents[0]/ r"data/mepsc"
paths = sorted(list(path.glob("*.json")))
exp_dict = {}
for p in paths:
    with open(p, "r") as rf:
        temp = json.load(rf)
        exp_dict[int(temp["acq_number"])] = temp
x_array = np.arange(len(exp_dict[153]["array"])) / 10

The first thing to do is look through your data just to see what it looks like. For reference the data in this tutorial is from a layer 5 cell in the ACC of a P16 mouse. 
- The recorded data is usually in pA, as is the case for this data.
- It can be hard to see the events. They are usually slow and small.
- The acquisitions hover around 70. This is the amount of current injected to keep the cell at the holding voltage which in this case is -70 mV.

In [ ]:
def explore_plots(index):
    plt.plot(
        x_array,
        exp_dict[index]["array"],
        label=f"Acquisition: {index}",
    )

    plt.ylabel("Current (pA)")

    plt.xlabel("Time (ms)")
    plt.legend(loc=(1.04, 0.5))


out = widgets.interact(
    explore_plots,
    index=widgets.IntSlider(
        value=0,
        min=min(exp_dict.keys()),
        max=max(exp_dict.keys()),
        step=1,
    ),
)

First, we will define some important features of the acquisition so that we can reuse the settings throughout the analysis. It is important to note the all the parameters are going to be in samples. The current files were recorded at 10000 Hz so we are multiply the time by 10.

In [ ]:
baseline_start = 0
baseline_end = 10
sample_rate = 10000